In [1]:
import os
import csv

csv_input_file = r"C:\Users\hmane\Desktop\folders\memes\ct_data\ig_fb_combined_race_lgbtq_01012016_07312023.csv"
base_output_file = 'CT_annotations_batch_{}-{}.csv'
image_base_folder_path = r"C:\Users\hmane\Desktop\label_studio_images\10k_batches\"  # This is the directory containing batch folders
github_base_url = "https://raw.githubusercontent.com/heranmane/label_studio_images/main/10k_batches"


In [8]:
import pandas as pd
df = pd.read_csv(csv_input_file)

c:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (15,25,30,34,39,40,41,42) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [18]:
import os
import csv

# Define paths and variables
csv_input_file = r"C:\Users\hmane\Desktop\folders\memes\ct_data\ig_fb_combined_race_lgbtq_01012016_07312023.csv"
image_base_folder_path = r"C:/Users/hmane/Desktop/label_studio_images/10k_batches/"  # This is the directory containing batch folders
base_output_file = 'CT_annotations_batch_{}.csv'
github_base_url = "https://raw.githubusercontent.com/heranmane/label_studio_images/main/10k_batches/"
# Read original CSV to get the text associated with images
text_data = {}
with open(csv_input_file, 'r', newline='', encoding='utf-8') as infile:
    csv_reader = csv.DictReader(infile)
    for row in csv_reader:
        post_url = row['URL']
        post_id = post_url.split("/")[-1]
        text_data[post_id] = row['Message']

# Process batches
batch_folders = [f for f in os.listdir(image_base_folder_path) if f.startswith('batch_')]
batch_folders.sort(key=lambda x: int(x.split('_')[1]))  # Sort folders numerically by batch number

# Create one CSV per batch
for batch in batch_folders:
    batch_path = os.path.join(image_base_folder_path, batch)
    output_csv_file = base_output_file.format(batch)

    with open(output_csv_file, 'w', newline='', encoding='utf-8') as outfile:
        fieldnames = ['batch', 'image', 'text']
        csv_writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        csv_writer.writeheader()

        for image_filename in os.listdir(batch_path):
            # Ensure the image filename follows the expected format
            parts = image_filename.split('-')
            if len(parts) > 1:
                post_id = parts[1].split('.')[0]
                if post_id in text_data:
                    image_url = github_base_url + os.path.join(batch, image_filename).replace('\\', '/')
                    new_row = {
                        'batch': batch,
                        'image': image_url,
                        'text': text_data[post_id]
                    }
                    csv_writer.writerow(new_row)

    print(f"CSV file created for batch {batch}: {output_csv_file}")

CSV file created for batch batch_17: CT_annotations_batch_batch_17.csv
CSV file created for batch batch_18: CT_annotations_batch_batch_18.csv
CSV file created for batch batch_19: CT_annotations_batch_batch_19.csv
CSV file created for batch batch_20: CT_annotations_batch_batch_20.csv
CSV file created for batch batch_21: CT_annotations_batch_batch_21.csv
CSV file created for batch batch_22: CT_annotations_batch_batch_22.csv
CSV file created for batch batch_23: CT_annotations_batch_batch_23.csv
CSV file created for batch batch_24: CT_annotations_batch_batch_24.csv
CSV file created for batch batch_25: CT_annotations_batch_batch_25.csv
CSV file created for batch batch_26: CT_annotations_batch_batch_26.csv
CSV file created for batch batch_27: CT_annotations_batch_batch_27.csv
CSV file created for batch batch_28: CT_annotations_batch_batch_28.csv
CSV file created for batch batch_29: CT_annotations_batch_batch_29.csv
CSV file created for batch batch_30: CT_annotations_batch_batch_30.csv
CSV fi

In [26]:
# import os

# def get_post_ids_and_image_names(base_path):
#     post_ids = {}
#     image_names = {}
#     batch_folders = []

#     # Iterate through all subfolders in the base path
#     for root, dirs, files in os.walk(base_path):
#         batch_name = os.path.basename(root)
#         if files:
#             batch_folders.append(batch_name)
#             for file in files:
#                 if file not in image_names:
#                     image_names[file] = batch_name
#                 else:
#                     image_names[file] += f", {batch_name}"
                
#                 parts = file.split('-')
#                 if len(parts) > 1:
#                     post_id = parts[1].split('.')[0]
#                     if post_id not in post_ids:
#                         post_ids[post_id] = batch_name
#                     else:
#                         post_ids[post_id] += f", {batch_name}"
    
#     return post_ids, image_names, batch_folders

# # Paths to the main folders
# folder1 = r"C:/Users/hmane/Desktop/label_studio_images/10k_batches/"
# folder2 = r"C:/Users/hmane/Desktop/label_studio_images/split_images/ct_week1/"

# # Get post_ids and image_names from both folders
# post_ids1, image_names1, batch_folders1 = get_post_ids_and_image_names(folder1)
# post_ids2, image_names2, batch_folders2 = get_post_ids_and_image_names(folder2)

# # # Print all batch folder names
# # print("Batch folders in folder1:")
# # for batch in batch_folders1:
# #     print(batch)

# # print("\nBatch folders in folder2:")
# # for batch in batch_folders2:
# #     print(batch)

# # Find duplicates
# duplicate_post_ids = set(post_ids1.keys()).intersection(set(post_ids2.keys()))
# duplicate_image_names = set(image_names1.keys()).intersection(set(image_names2.keys()))

# # Output results
# print(f"\nDuplicate post IDs: {len(duplicate_post_ids)} found.")
# for post_id in duplicate_post_ids:
#     print(f"{post_id} found in batches: {post_ids1[post_id]} and {post_ids2[post_id]}")

# print(f"\nDuplicate image names: {len(duplicate_image_names)} found.")
# for image_name in duplicate_image_names:
#     print(f"{image_name} found in batches: {image_names1[image_name]} and {image_names2[image_name]}")


In [27]:
import os
import csv

def count_csv_rows(directory):
    csv_files = [f for f in os.listdir(directory) if f.endswith('.csv')]
    for csv_file in csv_files:
        file_path = os.path.join(directory, csv_file)
        with open(file_path, 'r', newline='', encoding='utf-8') as infile:
            csv_reader = csv.reader(infile)
            row_count = sum(1 for row in csv_reader) - 1  # Subtract 1 to exclude the header
            print(f"CSV file: {csv_file}, Number of rows: {row_count}")

# Paths to the directories containing CSV files
folder1 = r"C:/Users/hmane/Desktop/label_studio_images/10k_batches/"
folder2 = r"C:/Users/hmane/Desktop/label_studio_images/split_images/ct_week1/"

# Print CSV file names and row counts in folder1
print("CSV files in folder1:")
count_csv_rows(folder1)

# Print CSV file names and row counts in folder2
print("\nCSV files in folder2:")
count_csv_rows(folder2)


CSV files in folder1:
CSV file: CT_annotations_batch_batch_17.csv, Number of rows: 300
CSV file: CT_annotations_batch_batch_18.csv, Number of rows: 300
CSV file: CT_annotations_batch_batch_19.csv, Number of rows: 300
CSV file: CT_annotations_batch_batch_20.csv, Number of rows: 300
CSV file: CT_annotations_batch_batch_21.csv, Number of rows: 300
CSV file: CT_annotations_batch_batch_22.csv, Number of rows: 300
CSV file: CT_annotations_batch_batch_23.csv, Number of rows: 300
CSV file: CT_annotations_batch_batch_24.csv, Number of rows: 300
CSV file: CT_annotations_batch_batch_25.csv, Number of rows: 300
CSV file: CT_annotations_batch_batch_26.csv, Number of rows: 300
CSV file: CT_annotations_batch_batch_27.csv, Number of rows: 300
CSV file: CT_annotations_batch_batch_28.csv, Number of rows: 300
CSV file: CT_annotations_batch_batch_29.csv, Number of rows: 300
CSV file: CT_annotations_batch_batch_30.csv, Number of rows: 300
CSV file: CT_annotations_batch_batch_31.csv, Number of rows: 300
CSV

In [28]:
# pd.read_csv(r'C:\Users\hmane\Desktop\label_studio_images\split_images\CT_annotations_batch_batch_47.csv')

create tracking sheet

In [17]:
import csv

# List of CSV files uploaded for annotation
csv_files = [
    "CT_annotations_batch_1-3.csv",
    "CT_annotations_batch_4-6.csv",
    "CT_annotations_batch_7-9.csv",
    "CT_annotations_batch_10-12.csv",
    "CT_annotations_batch_13-15.csv",
    "CT_annotations_batch_16.csv"
]

# Annotators information (example names/IDs)
annotators = [("Annotator1", "Annotator2", "TieBreaker")]

# Create the tracking CSV
tracking_csv = "annotation_tracking.csv"
with open(tracking_csv, 'w', newline='') as file:
    writer = csv.writer(file)
    # Write the header
    writer.writerow(["CSV File Name", "Annotator 1", "Annotator 1 Status", 
                     "Annotator 2", "Annotator 2 Status", "Tie Breaker", 
                     "Tie Breaker Status", "Overall Status"])
    
    # Write initial data
    for csv_file in csv_files:
        for annotator1, annotator2, tie_breaker in annotators:
            # Initial statuses can be adjusted as needed
            writer.writerow([csv_file, annotator1, "Not Started", annotator2, 
                             "Not Started", tie_breaker, "Not Started", "Incomplete"])

print("Tracking CSV created:", tracking_csv)


Tracking CSV created: annotation_tracking.csv


In [14]:
import pandas as pd 
df = pd.read_csv(r'C:\Users\hmane\Downloads\split_images\CT_annotations_batch_batch_1-batch_3.csv')
df

,batch,image,text
0,batch_1,https://raw.githubusercontent.com/heranmane/Me...,BLM Roundup & Ended Up at a Kill Lot in Texas!...
1,batch_1,https://raw.githubusercontent.com/heranmane/Me...,".Cherokee Speedway, we’re coming for ya! 👊 The..."
2,batch_1,https://raw.githubusercontent.com/heranmane/Me...,Marsha “Pay it No Mind” Johnson (1945-1992) wa...
3,batch_1,https://raw.githubusercontent.com/heranmane/Me...,🚨Taco Tuesday 🚨🌮🦐🌮 🚨2X1 Tacos with a Drink pur...
4,batch_1,https://raw.githubusercontent.com/heranmane/Me...,"Jacks Fam, please join us in welcoming Raegen ..."
...,...,...,...
295,batch_3,https://raw.githubusercontent.com/heranmane/Me...,Chili Cornbread Casserole https://ed.gr/duszu:...
296,batch_3,https://raw.githubusercontent.com/heranmane/Me...,HIGH WIND WARNING THRU 7 AM: A storm complex i...
297,batch_3,https://raw.githubusercontent.com/heranmane/Me...,The weather team is tracking a winter storm sy...
298,batch_3,https://raw.githubusercontent.com/heranmane/Me...,✨ NEW PRODUCT ALERT! ✨ We need your vote becau...


In [7]:
df2 = pd.read_csv(r'C:\Users\hmane\Downloads\split_images\CT_annotations_batch_batch_3-batch_5.csv')
df2

,batch,image,text
0,batch_3,https://raw.githubusercontent.com/heranmane/Me...,City officials expect an 80% chance of rain on...
1,batch_3,https://raw.githubusercontent.com/heranmane/Me...,Reposted from @prparadenyc We are currently pl...
2,batch_3,https://raw.githubusercontent.com/heranmane/Me...,"""We seek the history of Jewish women out of ou..."
3,batch_3,https://raw.githubusercontent.com/heranmane/Me...,The big guy got Caught.
4,batch_3,https://raw.githubusercontent.com/heranmane/Me...,On this special day honoring the 100th birthda...
...,...,...,...
295,batch_5,https://raw.githubusercontent.com/heranmane/Me...,Now introducing Iwate Prefecture! Iwate is kno...
296,batch_5,https://raw.githubusercontent.com/heranmane/Me...,Ever wondered about different Eggplants? Here'...
297,batch_5,https://raw.githubusercontent.com/heranmane/Me...,NaN
298,batch_5,https://raw.githubusercontent.com/heranmane/Me...,"Hey Folks! On Sunday, January 9 at 8pm ET, I w..."
